# LensPR Evaluation: With vs Without

This notebook demonstrates the difference between using Claude Code with LensPR tools vs traditional Read/Grep/Edit approach.

## Setup

In [1]:
import sys
sys.path.insert(0, '..')

import lenspr
import subprocess
from pathlib import Path

# Initialize LensPR on this project
PROJECT_ROOT = Path('..').resolve()
ctx = lenspr.init(str(PROJECT_ROOT))
print(f"Project: {PROJECT_ROOT}")
print(f"Initialized: {ctx is not None}")

Project: /Users/kyryloprymak/code/lenspr
Initialized: True


In [2]:
# Helper to call LensPR tools
def lens(tool_name: str, **params):
    """Call a LensPR tool and return data."""
    result = lenspr.handle_tool(f"lens_{tool_name}", params)
    if not result['success']:
        print(f"Error: {result['error']}")
        return None
    return result['data']

---
## Task 1: Understand a Function

**Goal:** Understand what `update_node` does and what depends on it.

### Traditional Approach (without LensPR)

In [3]:
# Traditional Step 1: Find the file containing the function
result = subprocess.run(
    ['grep', '-rn', 'def update_node', '../lenspr'],
    capture_output=True, text=True, cwd=str(PROJECT_ROOT / 'eval')
)
print("Step 1 - Find function definition:")
print(result.stdout)

Step 1 - Find function definition:
../lenspr/database.py:244:def update_node_source(node_id: str, new_source: str, new_hash: str, db_path: Path) -> bool:



In [4]:
# Traditional Step 2: Read the entire file to see the function
file_path = PROJECT_ROOT / 'lenspr/tools/modification.py'
with open(file_path) as f:
    content = f.read()
    lines = content.splitlines()
    print(f"File has {len(lines)} lines, {len(content)} characters")
    print("\nNeed to manually find function boundaries...")
    print("Or read the entire file into context.")

File has 354 lines, 11217 characters

Need to manually find function boundaries...
Or read the entire file into context.


In [5]:
# Traditional Step 3: Find callers with grep (noisy)
result = subprocess.run(
    ['grep', '-rn', 'update_node', '../lenspr', '../tests'],
    capture_output=True, text=True, cwd=str(PROJECT_ROOT / 'eval')
)
lines = result.stdout.strip().split('\n')
print(f"Step 3 - grep 'update_node' returned {len(lines)} matches:")
for line in lines[:15]:
    print(f"  {line[:100]}")
if len(lines) > 15:
    print(f"  ... and {len(lines) - 15} more")
print("\nProblem: includes definitions, imports, strings, comments - not just callers!")

Step 3 - grep 'update_node' returned 36 matches:
  ../lenspr/claude_tools.py:59:    handle_update_node as _handle_update_node,
  ../lenspr/claude_tools.py:94:    "_handle_update_node",
  ../lenspr/tools/modification.py:18:def handle_update_node(params: dict, ctx: LensContext) -> ToolRes
  ../lenspr/tools/__init__.py:38:    handle_update_node,
  ../lenspr/tools/__init__.py:67:    "handle_update_node",
  ../lenspr/tools/__init__.py:102:    "lens_update_node": ("lenspr.tools.modification", "handle_update
  Binary file ../lenspr/tools/__pycache__/schemas.cpython-312.pyc matches
  Binary file ../lenspr/tools/__pycache__/modification.cpython-312.pyc matches
  Binary file ../lenspr/tools/__pycache__/__init__.cpython-312.pyc matches
  ../lenspr/tools/schemas.py:85:        "name": "lens_update_node",
  ../lenspr/tools/schemas.py:104:            "WITHOUT actually applying changes. Use before lens_updat
  ../lenspr/database.py:244:def update_node_source(node_id: str, new_source: str, new_hash: st

**Traditional approach summary:**
- 3+ separate commands
- Read entire files
- Noisy output (definitions, imports, strings mixed with actual calls)
- No understanding of call graph
- No automatic test discovery

### LensPR Approach

In [ ]:
# LensPR: ONE call gets source + callers + callees + tests
data = lens('context', node_id='lenspr.tools.modification.update_node')

if data:
    print("=== TARGET SOURCE ===")
    source = data['target']['source']
    print(source[:600] + '...' if len(source) > 600 else source)

    print("\n=== CALLERS (what calls this function) ===")
    for caller in data.get('callers', []):
        print(f"  - {caller['id']}")

    print("\n=== CALLEES (what this function calls) ===")
    for callee in data.get('callees', [])[:10]:
        print(f"  - {callee['id']}")

    print("\n=== RELATED TESTS ===")
    for test in data.get('tests', []):
        print(f"  - {test['id']}")

Error: Node not found: lenspr.tools.modification.update_node


**LensPR approach summary:**
- 1 call
- Only the function source, not the whole file
- Clean, structured output (real callers, not grep noise)
- Graph-based understanding
- Automatic test discovery

---
## Task 2: Safe Refactoring - Know Impact Before Change

**Goal:** Rename a function and understand what will break.

### Traditional Approach

In [ ]:
# Traditional: grep + hope for the best
result = subprocess.run(
    ['grep', '-rl', 'get_node', '../lenspr'],
    capture_output=True, text=True, cwd=str(PROJECT_ROOT / 'eval')
)
files = [f for f in result.stdout.strip().split('\n') if f]
print(f"Files containing 'get_node': {len(files)}")
for f in files:
    print(f"  {f}")

print("\n⚠️  Problems:")
print("  - Includes 'lens_get_node', 'db_get_node' - not the same function!")
print("  - No way to know which are actual callers vs unrelated matches")
print("  - No severity assessment (CRITICAL? LOW?)")
print("  - No idea what tests will break")

### LensPR Approach

In [ ]:
# LensPR: Check impact BEFORE making any changes
data = lens('check_impact', node_id='lenspr.database.get_node', depth=2)

if data:
    print(f"🎯 Severity: {data['severity']}")
    print(f"📊 Direct dependents: {data['direct_count']}")
    print(f"📊 Total affected (depth=2): {data['total_affected']}")
    print()
    print("Affected nodes by depth:")
    for node in data.get('affected', [])[:15]:
        print(f"  [depth {node['depth']}] {node['id']} ({node['type']})")

In [ ]:
# LensPR: Find ALL usages - callers, importers, inheritors
data = lens('find_usages', node_id='lenspr.database.get_node')

if data:
    print("CALLERS (actually call this function):")
    for u in data.get('callers', []):
        print(f"  - {u}")

    print("\nIMPORTERS (import this function):")
    for u in data.get('importers', []):
        print(f"  - {u}")

    print(f"\nTEST USAGES: {len(data.get('test_usages', []))} tests use this")

---
## Task 3: Validate Changes Before Applying

**Goal:** Catch errors BEFORE they break the codebase.

In [ ]:
# Traditional: Make the change, run tests, hope for the best
print("Traditional workflow:")
print("  1. Edit file with sed/vim/IDE")
print("  2. Run tests")
print("  3. Tests fail? Debug and fix")
print("  4. Repeat until green")
print("\n  ⚠️  Feedback loop is SLOW - errors found after the fact")

In [ ]:
# LensPR: Validate BEFORE applying

# Bad code - adds a required parameter (breaks callers!)
bad_code = '''
def get_node(db_path: Path, node_id: str, required_new_param: str) -> Node | None:
    """Get a node by ID - but now with a breaking change!"""
    conn = sqlite3.connect(db_path)
    return None
'''

data = lens('validate_change', node_id='lenspr.database.get_node', new_source=bad_code)

if data:
    print(f"✅ Valid syntax: {data.get('valid', False)}")
    print(f"\n⚠️  Warnings:")
    for w in data.get('warnings', []):
        print(f"  - {w}")
    print(f"\n❌ Errors:")
    for e in data.get('errors', []):
        print(f"  - {e}")

---
## Task 4: Code Quality Metrics

In [ ]:
# LensPR: Health check - impossible with traditional tools
data = lens('health')

if data:
    print("📊 PROJECT HEALTH")
    print(f"  Nodes: {data['total_nodes']}")
    print(f"  Edges: {data['total_edges']}")
    print(f"  Confidence: {data['confidence_pct']}%")
    print(f"  Docstrings: {data['docstring_pct']}%")
    print(f"  Unresolved: {data['unresolved_count']} edges")
    print(f"  Circular imports: {len(data['circular_imports'])}")

In [ ]:
# LensPR: Find dead code - impossible to do reliably with grep
data = lens('dead_code')

if data:
    dead = data.get('dead_nodes', [])
    print(f"☠️  Potentially dead code: {len(dead)} functions")
    for node in dead[:10]:
        print(f"  - {node}")
    if len(dead) > 10:
        print(f"  ... and {len(dead) - 10} more")

---
## Task 5: Search with Graph Context

In [ ]:
# Traditional grep - just lines
result = subprocess.run(
    ['grep', '-rn', 'raise.*Error', '../lenspr'],
    capture_output=True, text=True, cwd=str(PROJECT_ROOT / 'eval')
)
lines = result.stdout.strip().split('\n')[:10]
print("Traditional grep 'raise.*Error':")
for line in lines:
    print(f"  {line[:80]}")
print("\n  ⚠️  No context about WHICH FUNCTION contains this line")

In [ ]:
# LensPR grep - shows containing function
data = lens('grep', pattern='raise.*Error', max_results=10)

if data:
    print("LensPR grep 'raise.*Error':")
    for match in data.get('matches', [])[:10]:
        print(f"  {match['file']}:{match['line']}")
        print(f"    → in: {match.get('containing_node', 'unknown')}")
        print(f"    → {match['content'][:60]}")
        print()

---
## Summary: Metrics Comparison

| Task | Traditional | LensPR | Improvement |
|------|-------------|--------|-------------|
| **Understand function** | 3+ grep/read calls | 1 `lens_context` | Fewer calls, structured output |
| **Find all usages** | grep (noisy) | `lens_find_usages` | Precise, graph-aware |
| **Know impact before change** | Impossible | `lens_check_impact` | Severity + affected list |
| **Find related tests** | Manual grep | Automatic | Built into `lens_context` |
| **Validate syntax** | Hope for best | `lens_validate_change` | Catch errors early |
| **Safe rename** | sed + pray | `lens_rename` | Updates all real references |
| **Find dead code** | Impossible | `lens_dead_code` | Graph reachability analysis |
| **Project health** | Impossible | `lens_health` | Confidence %, coverage |

---
## Conclusion

**LensPR provides:**
1. **Structured understanding** - One call gives source + callers + callees + tests
2. **Safe refactoring** - Know impact and severity before making changes
3. **Precise usages** - Graph-based, not text-based search
4. **Early validation** - Catch errors before applying changes
5. **Code quality** - Dead code detection, health metrics, confidence scores

**Traditional tools are:**
- Noisy (match strings, comments, unrelated code)
- Blind (no way to know impact before changing)
- Manual (need to piece together information from multiple sources)
- Reactive (find errors only after they happen)